In [1]:
import sys
import pandas as pd


sys.path.insert(0, '..')
summarizer_type = "openai"


topic_df = pd.read_parquet("gs://scraped-news-article-data-null/2023-topics-%s.parquet" % summarizer_type)
topic_df.head()

,topics,summary
0,0,The news articles revolve around the advanceme...
1,1,The news articles cover the fluctuations in oi...
2,2,The news articles revolve around the security ...
3,3,The Federal Reserve's interest rate decisions ...
4,4,The news articles cover various aspects of the...


In [2]:
topic_existing_sum = topic_df.loc[(topic_df.summary.str.len() > 0) & (topic_df.summary.str.lower().str.strip() != "no theme")]
topic_existing_sum.summary.head()

0    The news articles revolve around the advanceme...
1    The news articles cover the fluctuations in oi...
2    The news articles revolve around the security ...
3    The Federal Reserve's interest rate decisions ...
4    The news articles cover various aspects of the...
Name: summary, dtype: object

In [3]:
topic_existing_sum.summary.iloc[:10].to_list()

['The news articles revolve around the advancements and competition in the field of Artificial Intelligence (AI) and its integration into various industries. Companies like Google, Microsoft, OpenAI, and others are developing and launching AI-powered products like chatbots, virtual assistants, and generative AI. The articles also discuss the potential risks and concerns associated with AI.',
 "The news articles cover the fluctuations in oil prices and their causes, including OPEC+ output cuts, Fed rate hikes, US inflation data, and demand concerns. There are also mentions of China's economic growth, US oil activity, and the possibility of a rebound in oil prices.",
 'The news articles revolve around the security concerns and potential bans on TikTok, a Chinese-owned social media app, in various countries including the US, UK, Canada, Australia, New Zealand, Japan, and Austria. The articles also cover the efforts made by TikTok to address these concerns, including new data security regi

In [4]:
from summarizer.topic_sum import create_topic_filter

In [5]:
with open("/home/jupyter/apikey", "r") as api_fp:
    api_key = api_fp.read().strip()


filter_llm = create_topic_filter(api_key=api_key, temperature=0)
list(filter_llm(topic_existing_sum, "Which companies are making progress on artifical intelligence?"))

[RelevantTopic(topic_number='0', rating=1.0),
 RelevantTopic(topic_number='50', rating=0.4),
 RelevantTopic(topic_number='59', rating=1.0),
 RelevantTopic(topic_number='98', rating=0.6),
 RelevantTopic(topic_number='114', rating=0.6),
 RelevantTopic(topic_number='115', rating=0.4),
 RelevantTopic(topic_number='143', rating=0.4),
 RelevantTopic(topic_number='169', rating=0.8),
 RelevantTopic(topic_number='193', rating=0.8),
 RelevantTopic(topic_number='212', rating=0.7),
 RelevantTopic(topic_number='236', rating=0.7),
 RelevantTopic(topic_number='272', rating=0.2),
 RelevantTopic(topic_number='291', rating=0.6),
 RelevantTopic(topic_number='326', rating=0.4),
 RelevantTopic(topic_number='343', rating=0.6)]